In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_breast_cancer, fetch_california_housing, load_diabetes
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from scipy.stats import spearmanr, pearsonr

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import matplotlib.cm as cm

from sklearn.manifold import TSNE
import pandas as pd
import itertools

In [2]:
%config InlineBackend.figure_format='retina'

In [3]:
orthoG = "C:/Users/46705/Documents/SpiderSilk/data/annotations/Orthogroups_ls.tsv"
df_orthogroups = pd.read_csv(orthoG, sep='\t') 
first_last_columns = df_orthogroups.iloc[:, [0, -1]]

data_without_na = first_last_columns.dropna(subset=[first_last_columns.columns[-1]])
data_without_na
data_exploded = data_without_na.assign(last_column=data_without_na[data_without_na.columns[-1]].str.split(", ")).explode('last_column')

C:\Users\46705\AppData\Local\Temp\ipykernel_17640\2358411230.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,25

KeyboardInterrupt: 

In [4]:
path_lodings_mag_ara_sum =  "C:/Users/46705/Documents/SpiderSilk/output/loadings/Ara_MaG_sum.csv"
path_lodings_mag_ara_count =  "C:/Users/46705/Documents/SpiderSilk/output/loadings/ara_GC_PC3.csv"
path_lodings_mag_ara_bool_1 =  "C:/Users/46705/Documents/SpiderSilk/output/loadings/MaG_bool_PC1.csv"
path_lodings_mag_ara_bool_2 = "C:/Users/46705/Documents/SpiderSilk/output/loadings/MaG_bool_PC2.csv"
# path_lodings_all_sum =  "C:/Users/46705/Documents/SpiderSilk/output/loadings/all_sum.csv"
# path_lodings_ara_sum =  "C:/Users/46705/Documents/SpiderSilk/output/loadings/ara_sum_PC2.csv"

ara_mag_sum = pd.read_csv(path_lodings_mag_ara_sum, sep = ",")
ara_mag_GC = pd.read_csv(path_lodings_mag_ara_count, sep = ",")
# all_sum = pd.read_csv(path_lodings_all_sum, sep = ",")
# ara_sum = pd.read_csv(path_lodings_ara_sum, sep = ",")
mag_ara_bool_1 = pd.read_csv(path_lodings_mag_ara_bool_1, sep = ",")
mag_ara_bool_2 = pd.read_csv(path_lodings_mag_ara_bool_2, sep = ",")

proteomics_gland = "C:/Users/46705/Documents/SpiderSilk/data/proteomics/proteinsInGland.2024.csv"
prot_gland = pd.read_csv(proteomics_gland, sep = ",")

proteomics_silk = "C:/Users/46705/Documents/SpiderSilk/data/proteomics/proteinsInSilk.2024.csv"
prot_silk = pd.read_csv(proteomics_silk, sep = ",")

In [5]:
ara_mag_sum[ara_mag_sum["OG_id"] == "OG0000264"]

,OG_id,PC1,PC2,PC3,PC4,PC5,Group,GeneName,Product,Lascid,cluster
4,OG0000264,0.222516,-0.075873,0.128765,-0.161316,-0.055178,SpiCE,"Spice_4, Spice_9, Spice_15, Spice_3, Spice_12,...",hypothetical protein,"LASC001444,LASC001445,LASC001446,LASC001447,LA...",NaN


In [14]:
lista_top10 = list(ara_mag_sum[OG_id].head(10))

NameError: name 'OG_id' is not defined

## add OG to silkset

In [6]:
# silksetWog = prot_silk[["geneName", "product", "lascID", "scGeneName", "silkGroup"]]
silksetWog = prot_silk
merged_df = pd.merge(silksetWog, data_exploded[["last_column", "Orthogroup"]], how='left', left_on='lascID', right_on='last_column')

merged_df.drop(columns='last_column', inplace=True)
silk_og = merged_df


NameError: name 'data_exploded' is not defined

In [7]:
# silk_og.to_csv("C:/Users/46705/Documents/SpiderSilk/data/prot_silk_OG.csv", index = False)
spice_OG =silk_og[silk_og["silkGroup"]== "Spice-LMa"][["geneNameProtein", "scGeneName", "Orthogroup"]]["Orthogroup"]
# silk_og.to_csv("C:/Users/46705/Documents/SpiderSilk/data/prot_silk_OG.csv", index = False)
silk_og[silk_og["silkGroup"]== "Spice-LMa"][["geneNameProtein", "scGeneName", "Orthogroup"]]

,geneNameProtein,scGeneName,Orthogroup
10,SpiCE-LMa1,LASC013229,OG0009245
11,SpiCE-LMa2,xync,OG0013923
13,SpiCE-LMa3,LASC000319,OG0000171
19,SpiCE-LMa4,tpo-pxt,OG0000494
22,SpiCE-LMa6,LASC009843,OG0006927
23,SpiCE-LMa5,Spice-8,OG0000264


In [8]:
spice_OG_values = set(spice_OG)

matching_values = ara_mag_sum[ara_mag_sum["OG_id"].isin(spice_OG_values)]["OG_id"]

matching_values

4     OG0000264
44    OG0006927
79    OG0000171
Name: OG_id, dtype: object

In [9]:
# silk_og["Orthogroup"].unique() == ara_mag_sum["OG_id"]



silk_og_unique = set(silk_og["Orthogroup"].unique())
print(len(silk_og_unique))
ara_mag_sum_og_ids = set(ara_mag_sum["OG_id"])
common_values = silk_og_unique.intersection(ara_mag_sum_og_ids)
common_values_count = len(common_values)

print("Number of unique values in silk_og['Orthogroup'] also present in ara_mag_sum['OG_id']:", common_values_count)



150
Number of unique values in silk_og['Orthogroup'] also present in ara_mag_sum['OG_id']: 20


In [10]:
from scipy.stats import fisher_exact

observed = [[2, 4], [20-2, 173-40]]  
odds_ratio, p_value = fisher_exact(observed)

print("Odds ratio:", odds_ratio)
print("p-value:", p_value)


Odds ratio: 3.6944444444444446
p-value: 0.16911955929831424


In [12]:
list(ara_mag_sum["OG_id"].head(10))

['OG0000282',
 'OG0008243',
 'OG0008607',
 'OG0008312',
 'OG0000264',
 'OG0004717',
 'OG0007747',
 'OG0007860',
 'OG0004409',
 'OG0000189']

In [13]:
ara_mag_sum

,OG_id,PC1,PC2,PC3,PC4,PC5,Group,GeneName,Product,Lascid,cluster
0,OG0000282,-0.480160,0.200225,-0.166053,-0.072299,0.005558,NaN,"LASC013610, cd74_2, LASC013607, tg_5, LASC0136...","hypothetical protein,Thyroglobulin,Nidogen-2,H...","LASC013595,LASC013596,LASC013597,LASC013602,LA...",NaN
1,OG0008243,-0.398308,0.032694,0.283644,-0.469987,-0.127325,NaN,sp-1339#prd34368.1_4,Sp-1339#PRD34368.1 protein,LASC010015,NaN
2,OG0008607,0.232044,0.343374,-0.385207,-0.163741,-0.549446,NaN,LASC012976,hypothetical protein,LASC012976,"Duct_1, Duct_2"
3,OG0008312,-0.227529,-0.017233,-0.135446,0.112192,-0.006052,NaN,LASC018595,hypothetical protein,LASC018595,ZoneC_SpiCE-LMa1
4,OG0000264,0.222516,-0.075873,0.128765,-0.161316,-0.055178,SpiCE,"Spice_4, Spice_9, Spice_15, Spice_3, Spice_12,...",hypothetical protein,"LASC001444,LASC001445,LASC001446,LASC001447,LA...",NaN
...,...,...,...,...,...,...,...,...,...,...,...
168,OG0007089,0.001582,-0.057665,-0.027281,-0.023315,-0.007352,NaN,LASC003517,hypothetical protein,LASC003517,NaN
169,OG0006013,0.001425,-0.093077,-0.039545,-0.020985,0.062993,NaN,timm23,Mitochondrial import inner membrane translocas...,LASC017275,NaN
170,OG0004738,0.000700,-0.178953,-0.016423,0.064668,-0.095063,NaN,LASC004484,hypothetical protein,LASC004484,Duct_2
171,OG0003918,-0.000696,-0.020372,-0.012682,-0.039044,0.082980,NaN,rpl19,60S ribosomal protein L19,LASC009569,NaN


## check how many of 20 OG can be found in silkset

In [11]:
common = pd.merge(ara_mag_sum.head(20), silk_og, left_on ="OG_id", right_on ="Orthogroup" , how = "inner")
common

,OG_id,PC1,PC2,PC3,PC4,PC5,Group_x,GeneName,Product,Lascid,...,lascID,geneName_GOI,Nephila_genome_Class,Group_y,scGeneName,sc_gene_id,Length,geneNameProtein,silkGroup,Orthogroup
0,OG0000264,0.222516,-0.075873,0.128765,-0.161316,-0.055178,SpiCE,"Spice_4, Spice_9, Spice_15, Spice_3, Spice_12,...",hypothetical protein,"LASC001444,LASC001445,LASC001446,LASC001447,LA...",...,LASC007774,Spice_8,Spice_NMa4,SpiCE,Spice-8,maker-000078F-est-gff-StringTie-gene-26.2,147.0,SpiCE-LMa5,Spice-LMa,OG0000264
1,OG0000264,0.222516,-0.075873,0.128765,-0.161316,-0.055178,SpiCE,"Spice_4, Spice_9, Spice_15, Spice_3, Spice_12,...",hypothetical protein,"LASC001444,LASC001445,LASC001446,LASC001447,LA...",...,LASC014000,Spice_15,Spice_NMa4,SpiCE,Spice-15,maker-000197F-est-gff-StringTie-gene-10.0,NaN,NaN,NaN,OG0000264
2,OG0000189,-0.136838,0.049264,-0.065472,0.100008,0.003191,NaN,"pxd_pxt_3, pxt_2, pxd_5, pxd_pxt_4, pxd_pxt, p...","hypothetical protein,Chorion peroxidase,Peroxi...","LASC015555,LASC015557,LASC015558,LASC015560,LA...",...,LASC015562,NaN,NaN,NaN,pxd-4,maker-000238F-augustus-gene-13.41,NaN,NaN,NaN,OG0000189
3,OG0000369,-0.120817,0.050641,-0.029833,0.124924,-0.036854,NaN,"gld_15, LASC008285, gld_3, gld_19, beta_3","Glucose dehydrogenase [FAD%2C quinone],hypothe...","LASC001976,LASC008285,LASC008287,LASC020658,LA...",...,LASC008287,NaN,NaN,NaN,beta-3,evm-000086F-processed-gene-4.0,NaN,NaN,NaN,OG0000369
4,OG0002778,-0.097265,-0.012688,-0.018355,0.121373,-0.026275,Spidroin,Spidroin3,"Uncharacterized, protein, (Fragment)",LASC020272,...,LASC020272,Spidroin3,Unclassified,Spidroin,Spidroin3,maker-000428F-est-gff-StringTie-gene-4.8,2350.0,AmSp-like2,AmSp-like,OG0002778
5,OG0007367,-0.093221,-0.113635,-0.170399,-0.185069,0.222619,NaN,sec61b,Protein transport protein Sec61 subunit beta,LASC021718,...,LASC021718,NaN,NaN,NaN,sec61b,maker-000540F-augustus-gene-2.1,NaN,NaN,NaN,OG0007367


In [ ]:

# path_tail_sac = "C:/Users/46705/Documents/SpiderSilk/data/post_filtering/tail_sac_log_multisp.csv"
path_max = "C:/Users/46705/Documents/SpiderSilk/data/post_filtering/max_tail_sac_log_multisp.csv"
## Full dataset
# path = "C:/Users/46705/Documents/SpiderSilk/data/post_filtering/filtered_multisp_log.csv"


# path = "C:/Users/46705/Documents/SpiderSilk/data/post_filtering/filtered_full_multisp_log_20_lascID.csv"
# path = "C:/Users/46705/Documents/SpiderSilk/generalized/filtered_sp_added_90p_nonzero_values_log_multsp.csv"
path_anno = "C:/Users/46705/Documents/SpiderSilk/data/annotations/annotation.csv"
# path_not_log = "C:/Users/46705/Documents/SpiderSilk/generalized/filtered_sp_added_90p_nonzero_values.csv"
# df = pd.read_csv(path, sep= ",")


path_MaG = "C:/Users/46705/Documents/SpiderSilk/data/post_filtering/MaG_ara.csv"

mag =pd.read_csv(path_MaG, sep= ",")

file_connectoion = "C:/Users/46705/Documents/SpiderSilk/data/raw_data/S1-S4/data_s1.csv"

df_anno = pd.read_csv(path_anno, sep= ",")

spidroin_conts_path = "C:/Users/46705/Documents/SpiderSilk/data/pre_filtering/df_grouped.csv"

path_anno_ = "C:/Users/46705/Documents/SpiderSilk/data/annotations/OG_annotations.csv"

anno__ =pd.read_csv(path_anno_, sep= ",")


anno_spidroins = anno__[anno__['Group'].isin(["Spidroin", "SpiCE"])]
anno_spidroins

,Unnamed: 0,Group,geneName,product,lascID
85,OG0000086,SpiCE,"LASC011985,Spice_18,awa1,tnxb_7,LASC011991,LAS...","hypothetical protein,Cell wall protein AWA1,Te...","LASC011985,LASC011987,LASC011989,LASC011990,LA..."
262,OG0000264,SpiCE,"Spice_3,Spice_4,Spice_5,Spice_6,Spice_2,Spice_...",hypothetical protein,"LASC001444,LASC001445,LASC001446,LASC001447,LA..."
1269,OG0001273,SpiCE,"LASC001401,LASC001402,LASC001457,Spice_17",hypothetical protein,"LASC001401,LASC001402,LASC001457,LASC019594"
2194,OG0002201,SpiCE,Spice_19,"Tyrosine-protein kinase receptor Tie-1,Multipl...",LASC017498
165894,OG0000050,Spidroin,"MaSp4,MaSp2a,MaSp2b,MaSp2c,MaSp1a,MaSp1b,MaSp1...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...","LASC004263,LASC004265,LASC004267,LASC004269,LA..."
165895,OG0000139,Spidroin,"AcSp1b,AcSp1c","hypothetical, protein","LASC012314,LASC012315"
165896,OG0000344,Spidroin,"TuSpa,TuSpb","Tubuliform, spidroin, 1, variant, 1, (Fragment)","LASC002331,LASC008470"
165897,OG0000846,Spidroin,"AgSp2a,AgSp2b,agsp-c#prd26655.1","AgSp-b#PRD22063.1 protein,AgSp-a#PRD23399.1 pr...","LASC004210,LASC004211,LASC004212"
165898,OG0001354,Spidroin,AgSp1,AgSp-c#PRD26655.1 protein,LASC014453
165899,OG0001803,Spidroin,FlSp-like,Sp-5803#PRD29580.1 protein,LASC000543


### Decide on what df to look at. 


## Agreeing OG of high loadings across datasets

In [ ]:
df_list = [ara_mag_sum, mag_ara_bool_1, mag_ara_bool_2, ara_mag_GC]
df_names = ['ara_mag_sum','mag_ara_bool_1', 'mag_ara_bool_2', "ara_mag_GC"]

In [ ]:
# # Sample data
# df_list = [ara_mag_sum, ara_sum, all_sum, mag_ara_bool_1, mag_ara_bool_2]
# df_names = ['ara_mag_sum', 'ara_sum', 'all_sum', 'mag_ara_bool_1', 'mag_ara_bool_2']

def find_common_og_ids(df1, df2):
    merged_df = pd.merge(df1.head(20), df2.head(20), on='OG_id', how='inner')
    return list(merged_df["OG_id"])

common_og_ids_dict = {}

combos = list(itertools.combinations(range(len(df_list)), 2))
for combo in combos:
    df1_index, df2_index = combo
    df1 = df_list[df1_index]
    df2 = df_list[df2_index]
    common_og_ids = find_common_og_ids(df1, df2)
    common_og_ids_dict[(df_names[df1_index], df_names[df2_index])] = common_og_ids

matrix = pd.DataFrame("", index=df_names, columns=df_names)

for combo, og_ids in common_og_ids_dict.items():
    df1_name, df2_name = combo
    matrix.loc[df1_name, df2_name] = og_ids
    matrix.loc[df2_name, df1_name] = og_ids 

matrix


,ara_mag_sum,mag_ara_bool_1,mag_ara_bool_2,ara_mag_GC
ara_mag_sum,,"[OG0000282, OG0008243, OG0008607, OG0008312, O...","[OG0000282, OG0008243, OG0008312, OG0007860, O...","[OG0000282, OG0000264, OG0000369, OG0000442, O..."
mag_ara_bool_1,"[OG0000282, OG0008243, OG0008607, OG0008312, O...",,"[OG0002778, OG0008312, OG0008243, OG0002319, O...",[OG0000282]
mag_ara_bool_2,"[OG0000282, OG0008243, OG0008312, OG0007860, O...","[OG0002778, OG0008312, OG0008243, OG0002319, O...",,[OG0000282]
ara_mag_GC,"[OG0000282, OG0000264, OG0000369, OG0000442, O...",[OG0000282],[OG0000282],


In [ ]:

matrix_counts = pd.DataFrame(0, index=df_names, columns=df_names)

for combo, og_ids in common_og_ids_dict.items():
    df1_name, df2_name = combo
    count = len(og_ids)
    matrix_counts.loc[df1_name, df2_name] = count
    matrix_counts.loc[df2_name, df1_name] = count

matrix_counts


,ara_mag_sum,mag_ara_bool_1,mag_ara_bool_2,ara_mag_GC
ara_mag_sum,0,7,7,5
mag_ara_bool_1,7,0,13,1
mag_ara_bool_2,7,13,0,1
ara_mag_GC,5,1,1,0


In [ ]:
unique_values = list(prot_gland['geneNameProtein'].dropna().unique())
prot18 = prot_gland[prot_gland['geneNameProtein'].isin(unique_values)]
prot18 = pd.merge(prot18, prot_silk[["lascID", "geneNameProtein"]], on= "geneNameProtein", how= "left")
# anno_exploded = anno__['lascID'].str.split(',').explode().astype(str)
anno_exploded = anno__.assign(lascID=anno__['lascID'].str.split(','))
anno_exploded = anno_exploded.explode('lascID')
anno_exploded[anno_exploded["lascID"]== "LASC007774"]


merged_df = pd.merge(prot18, anno_exploded, how='left', on='lascID')
merged_df.rename(columns={'Unnamed: 0': 'Ortogroup'}, inplace=True)
merged_df
prot18_ = merged_df[["lascID", "Ortogroup", "geneNameProtein"]]

In [ ]:
anno_exploded[anno_exploded["lascID"]== "LASC007774"]

,Unnamed: 0,Group,geneName,product,lascID
262,OG0000264,SpiCE,"Spice_3,Spice_4,Spice_5,Spice_6,Spice_2,Spice_...",hypothetical protein,LASC007774


## OGs of high loadings also found in silk of gland proteomics data'


In [ ]:
pc = "PC1"
for df_type in df_list: 
    for pc in ["PC1"]: 
        
        df = df_type.reindex(df_type[pc].abs().sort_values(ascending=False).index)
        def check_prot18(row):
            values = row['Lascid'].split(',')
            if any(value in prot18_['lascID'].values for value in values):
                return 'Yes'
            else:
                return '-'

        def check_prot_silk(row):
            values = row['Lascid'].split(',')
            if any(value in prot_silk['lascID'].values for value in values):
                return 'Yes'
            else:
                return '-'

        def check_gland_silk(row):
            values = row['GeneName'].split(',')
            if any(value in prot_gland['geneName'].values for value in values):
                return 'Yes'
            else:
                return '-'

        df = ara_mag_sum.reindex(ara_mag_sum[pc].abs().sort_values(ascending=False).index)


        create_full_info_df = df[['OG_id', pc, 'Group', 'GeneName',
            'Product', 'Lascid', 'cluster']]

        create_full_info_df['18protein dataset'] = create_full_info_df.apply(check_prot18, axis=1)
        create_full_info_df['silk proteomics'] = create_full_info_df.apply(check_prot_silk, axis=1)
        create_full_info_df['gland proteomics'] = create_full_info_df.apply(check_gland_silk, axis=1)

create_full_info_df[['OG_id',pc, 'GeneName', 'cluster',
        '18protein dataset', 'silk proteomics', 'gland proteomics' ]]
        



C:\Users\46705\AppData\Local\Temp\ipykernel_7688\2271155724.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_full_info_df['18protein dataset'] = create_full_info_df.apply(check_prot18, axis=1)
C:\Users\46705\AppData\Local\Temp\ipykernel_7688\2271155724.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_full_info_df['silk proteomics'] = create_full_info_df.apply(check_prot_silk, axis=1)
C:\Users\46705\AppData\Local\Temp\ipykernel_7688\2271155724.py:38: SettingWithCopyWarning: 
A value is 

,OG_id,PC1,GeneName,cluster,18protein dataset,silk proteomics,gland proteomics
0,OG0000282,-0.480160,"LASC013610, cd74_2, LASC013607, tg_5, LASC0136...",NaN,-,-,-
1,OG0008243,-0.398308,sp-1339#prd34368.1_4,NaN,-,-,-
2,OG0008607,0.232044,LASC012976,"Duct_1, Duct_2",-,-,-
3,OG0008312,-0.227529,LASC018595,ZoneC_SpiCE-LMa1,-,-,-
4,OG0000264,0.222516,"Spice_4, Spice_9, Spice_15, Spice_3, Spice_12,...",NaN,Yes,Yes,-
...,...,...,...,...,...,...,...
168,OG0007089,0.001582,LASC003517,NaN,-,-,Yes
169,OG0006013,0.001425,timm23,NaN,-,-,-
170,OG0004738,0.000700,LASC004484,Duct_2,-,-,-
171,OG0003918,-0.000696,rpl19,NaN,-,-,Yes


In [ ]:
pc = "PC1"
for df_type in df_list: 
    for pc in ["PC1"]: 
        
        df = df_type.reindex(df_type[pc].abs().sort_values(ascending=False).index).head(20)

        def check_prot18(row):
            values = row['Lascid'].split(',')
            if any(value in prot18_['lascID'].values for value in values):
                return 'Yes'
            else:
                return '-'

        def check_prot_silk(row):
            values = row['Lascid'].split(',')
            if any(value in prot_silk['lascID'].values for value in values):
                return 'Yes'
            else:
                return '-'

        def check_gland_silk(row):
            values = row['GeneName'].split(',')
            if any(value in prot_gland['geneName'].values for value in values):
                return 'Yes'
            else:
                return '-'

        df = ara_mag_sum.reindex(ara_mag_sum[pc].abs().sort_values(ascending=False).index).head(20)


        create_full_info_df = df.head(20)[['OG_id', pc, 'Group', 'GeneName',
            'Product', 'Lascid', 'cluster']]

        create_full_info_df['18protein dataset'] = create_full_info_df.apply(check_prot18, axis=1)
        create_full_info_df['silk proteomics'] = create_full_info_df.apply(check_prot_silk, axis=1)
        create_full_info_df['gland proteomics'] = create_full_info_df.apply(check_gland_silk, axis=1)

info_ = create_full_info_df[['OG_id',pc,'GeneName', "cluster", 
        '18protein dataset', 'silk proteomics']]



common = pd.merge(info_, silk_og[["geneNameProtein", "Orthogroup"]], left_on ="OG_id", right_on ="Orthogroup" , how = "left").iloc[:, :-1]
# common[['OG_id', 'PC1', 'GeneName','geneNameProtein', '18protein dataset', 'silk proteomics']]
df_grouped = common.groupby('OG_id').agg({
    'PC1': "first",  
    'GeneName': lambda x: ','.join(x.astype(str)), 
    'geneNameProtein': lambda x: ','.join(x.astype(str)), 
    'cluster': lambda x: ','.join(x.astype(str)), 
    '18protein dataset': "first", 
    'silk proteomics': "first"
}).reset_index()

df_grouped = df_grouped.reindex(df_grouped['PC1'].abs().sort_values(ascending=False).index)
df_grouped

# create_full_info_df[['OG_id',pc,'GeneName', 'cluster',
        # '18protein dataset', 'silk proteomics', 'gland proteomics' ]]
latex_table = df_grouped.to_latex(index=False)

print(latex_table)

\begin{tabular}{lrlllll}
\toprule
OG_id & PC1 & GeneName & geneNameProtein & cluster & 18protein dataset & silk proteomics \\
\midrule
OG0000282 & -0.480160 & LASC013610, cd74_2, LASC013607, tg_5, LASC013609, LASC013597, LASC013604, LASC013605, LASC013608, nid2, LASC013602, LASC013595 & nan & nan & - & - \\
OG0008243 & -0.398308 & sp-1339#prd34368.1_4 & nan & nan & - & - \\
OG0008607 & 0.232044 & LASC012976 & nan & Duct_1, Duct_2 & - & - \\
OG0008312 & -0.227529 & LASC018595 & nan & ZoneC_SpiCE-LMa1 & - & - \\
OG0000264 & 0.222516 & Spice_4, Spice_9, Spice_15, Spice_3, Spice_12, Spice_13, Spice_1, Spice_11, Spice_6, Spice_8, Spice_14, Spice_10, Spice_5, Spice_2, Spice_7, Spice_16,Spice_4, Spice_9, Spice_15, Spice_3, Spice_12, Spice_13, Spice_1, Spice_11, Spice_6, Spice_8, Spice_14, Spice_10, Spice_5, Spice_2, Spice_7, Spice_16 & SpiCE-LMa5,nan & nan,nan & Yes & Yes \\
OG0004717 & -0.195772 & LASC001730, LASC001732, LASC001731, LASC001733 & nan & Duct_1, nan, Duct_2 & - & - \\
OG0007747

In [ ]:
prot18_["Ortogroup"].unique()

array(['OG0000050', 'OG0009245', 'OG0013923', 'OG0000171', 'OG0002778',
       'OG0007640', 'OG0000494', 'OG0000264', 'OG0006927'], dtype=object)

In [ ]:
silk_og[silk_og[["silkGroup", "Orthogroup"]]["Orthogroup"]== "OG0000264"]

,geneName,HFIP,LiBr,Urea,gene_id,Chr,Start,End,product,uniprot_id,lascID,geneName_GOI,Nephila_genome_Class,Group,scGeneName,sc_gene_id,Length,geneNameProtein,silkGroup,Orthogroup
23,Spice_8,0.160635,0.014399,0.153889,maker-000078F-est_gff_StringTie-gene-26.2,000078F,6151623,6170329,hypothetical protein,NaN,LASC007774,Spice_8,Spice_NMa4,SpiCE,Spice-8,maker-000078F-est-gff-StringTie-gene-26.2,147.0,SpiCE-LMa5,Spice-LMa,OG0000264
45,Spice_15,0.078117,0.000000,0.059856,maker-000197F-est_gff_StringTie-gene-10.0,000197F,2311869,2326978,hypothetical protein,NaN,LASC014000,Spice_15,Spice_NMa4,SpiCE,Spice-15,maker-000197F-est-gff-StringTie-gene-10.0,NaN,NaN,NaN,OG0000264


In [ ]:
import pandas as pd

counts = {'PC': [], '18 Protein': [], 'Silk Proteomics': [], 'Gland Proteomics': []}

for pc in ["PC1", "PC2", "PC3", "PC4", "PC5"]:
    print(pc)
    df = ara_mag_sum.reindex(ara_mag_sum[pc].abs().sort_values(ascending=False).index)
    
    def check_prot18(row):
        values = row['Lascid'].split(',')
        if any(value in prot18_['lascID'].values for value in values):
            return 'Yes'
        else:
            return 'No'

    def check_prot_silk(row):
        values = row['Lascid'].split(',')
        if any(value in prot_silk['lascID'].values for value in values):
            return 'Yes'
        else:
            return 'No'

    def check_gland_silk(row):
        values = row['GeneName'].split(',')
        if any(value in prot_gland['geneName'].values for value in values):
            return 'Yes'
        else:
            return 'No'


    create_full_info_df = df[['OG_id', 'Lascid', 'GeneName']]
    
    create_full_info_df['18protein dataset'] = create_full_info_df.apply(check_prot18, axis=1)
    create_full_info_df['silk proteomics'] = create_full_info_df.apply(check_prot_silk, axis=1)
    create_full_info_df['gland proteomics'] = create_full_info_df.apply(check_gland_silk, axis=1)
    

    count_yes_18protein = (create_full_info_df["18protein dataset"] == "Yes").sum()
    count_yes_silk_proteomics = (create_full_info_df["silk proteomics"] == "Yes").sum()
    count_yes_gland_proteomics = (create_full_info_df["gland proteomics"] == "Yes").sum()
    
    counts['PC'].append(pc)
    counts['18 Protein'].append(count_yes_18protein)
    counts['Silk Proteomics'].append(count_yes_silk_proteomics)
    counts['Gland Proteomics'].append(count_yes_gland_proteomics)
counts_df = pd.DataFrame(counts)

counts_df

PC1


C:\Users\46705\AppData\Local\Temp\ipykernel_7688\1755182338.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_full_info_df['18protein dataset'] = create_full_info_df.apply(check_prot18, axis=1)
C:\Users\46705\AppData\Local\Temp\ipykernel_7688\1755182338.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_full_info_df['silk proteomics'] = create_full_info_df.apply(check_prot_silk, axis=1)
C:\Users\46705\AppData\Local\Temp\ipykernel_7688\1755182338.py:41: SettingWithCopyWarning: 
A value is 

PC2
PC3


C:\Users\46705\AppData\Local\Temp\ipykernel_7688\1755182338.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_full_info_df['18protein dataset'] = create_full_info_df.apply(check_prot18, axis=1)
C:\Users\46705\AppData\Local\Temp\ipykernel_7688\1755182338.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_full_info_df['silk proteomics'] = create_full_info_df.apply(check_prot_silk, axis=1)
C:\Users\46705\AppData\Local\Temp\ipykernel_7688\1755182338.py:41: SettingWithCopyWarning: 
A value is 

PC4
PC5


C:\Users\46705\AppData\Local\Temp\ipykernel_7688\1755182338.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_full_info_df['silk proteomics'] = create_full_info_df.apply(check_prot_silk, axis=1)
C:\Users\46705\AppData\Local\Temp\ipykernel_7688\1755182338.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_full_info_df['gland proteomics'] = create_full_info_df.apply(check_gland_silk, axis=1)


,PC,18 Protein,Silk Proteomics,Gland Proteomics
0,PC1,6,20,91
1,PC2,6,20,91
2,PC3,6,20,91
3,PC4,6,20,91
4,PC5,6,20,91


In [ ]:
create_full_info_df

,OG_id,Lascid,GeneName,18protein dataset,silk proteomics,gland proteomics
2,OG0008607,LASC012976,LASC012976,No,No,No
11,OG0003565,"LASC006841,LASC016362,LASC019275","LASC016362, LASC019275, LASC006841",No,No,No
36,OG0005938,LASC000884,rpl21,No,No,Yes
16,OG0007367,LASC021718,sec61b,No,Yes,Yes
19,OG0000048,"LASC001511,LASC003211,LASC005802,LASC005803,LA...","LASC007766, LASC015501, LASC011062, LASC022435...",No,No,No
...,...,...,...,...,...,...
75,OG0001429,"LASC010675,LASC021902","rpl9_2, rpl9",No,No,Yes
143,OG0004676,LASC019543,tsnax,No,No,No
105,OG0003221,LASC011010,phgdh,No,No,Yes
103,OG0006712,LASC003348,kdelr2,No,No,No


In [ ]:
df = ara_mag_sum.reindex(ara_mag_sum['PC1'].abs().sort_values(ascending=False).index).head(20)
df_20 =df

df_20['GeneName'] = df_20['GeneName'].str.split(',')
df_20_exploded = df_20.explode('GeneName')

# joined_full_gland = pd.merge(prot_gland, df_20_exploded, left_on='geneName', right_on='GeneName', how='inner')
joined_gland = pd.merge(prot_gland, df_20_exploded, left_on='geneName', right_on='GeneName', how='inner')
print(f"{df_names[i]}, to gland proteomics")
print(joined_gland["OG_id"].unique())

df_20 = df.head(20)

df_20['Lascid'] = df_20['Lascid'].str.split(',')
df_20_exploded = df_20.explode('Lascid')

joined_silk = pd.merge(prot_silk, df_20_exploded, left_on='lascID', right_on='Lascid', how='inner')
print(f"{df_names[i]}, to silk proteomics")
print(joined_silk["OG_id"].unique())
joined_silk[['OG_id', "Lascid", "cluster", "Group_y", "geneName", "scGeneName", "geneName_GOI"]]

NameError: name 'i' is not defined

In [ ]:
for i, df in enumerate(df_list): 
    df_20 = df.head(20)

    df_20['GeneName'] = df_20['GeneName'].str.split(',')
    df_20_exploded = df_20.explode('GeneName')
    
    # joined_full_gland = pd.merge(prot_gland, df_20_exploded, left_on='geneName', right_on='GeneName', how='inner')
    joined_gland = pd.merge(prot_gland, df_20_exploded, left_on='geneName', right_on='GeneName', how='inner')
    print(f"{df_names[i]}, to gland proteomics")
    print(joined_gland["OG_id"].unique())

    df_20 = df.head(20)

    df_20['Lascid'] = df_20['Lascid'].str.split(',')
    df_20_exploded = df_20.explode('Lascid')

    joined_silk = pd.merge(prot_silk, df_20_exploded, left_on='lascID', right_on='Lascid', how='inner')
    print(f"{df_names[i]}, to silk proteomics")
    print(joined_silk["OG_id"].unique())



C:\Users\46705\AppData\Local\Temp\ipykernel_6580\19846268.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_20['GeneName'] = df_20['GeneName'].str.split(',')
C:\Users\46705\AppData\Local\Temp\ipykernel_6580\19846268.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_20['Lascid'] = df_20['Lascid'].str.split(',')
C:\Users\46705\AppData\Local\Temp\ipykernel_6580\19846268.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

ara_mag_sum, to gland proteomics
['OG0002778' 'OG0007747' 'OG0003330' 'OG0005788' 'OG0007367' 'OG0007860'
 'OG0004409']
ara_mag_sum, to silk proteomics
['OG0002778' 'OG0000369' 'OG0000264' 'OG0000189' 'OG0007367']
mag_ara_bool_1, to gland proteomics
['OG0004629' 'OG0002778' 'OG0008612' 'OG0007640' 'OG0009320' 'OG0002604'
 'OG0008177' 'OG0008600' 'OG0005938' 'OG0004409' 'OG0002039']
mag_ara_bool_1, to silk proteomics
['OG0002778' 'OG0007640' 'OG0002604']
mag_ara_bool_2, to gland proteomics
['OG0004629' 'OG0002778' 'OG0003330' 'OG0008612' 'OG0009320' 'OG0004318'
 'OG0002604' 'OG0007860' 'OG0008600' 'OG0005938' 'OG0002319' 'OG0002039']
mag_ara_bool_2, to silk proteomics
['OG0002778' 'OG0006927' 'OG0004318' 'OG0002604']
ara_mag_GC, to gland proteomics
['OG0000050' 'OG0000055' 'OG0000369' 'OG0000609' 'OG0000073' 'OG0000022'
 'OG0000081' 'OG0000127']
ara_mag_GC, to silk proteomics
['OG0000050' 'OG0000369' 'OG0000264' 'OG0000078']


C:\Users\46705\AppData\Local\Temp\ipykernel_6580\19846268.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_20['Lascid'] = df_20['Lascid'].str.split(',')


In [ ]:
joined_silk

,geneName,HFIP,LiBr,Urea,gene_id,Chr,Start,End,product,uniprot_id,...,PC1,PC2,PC3,PC4,PC5,Group_y,GeneName,Product,Lascid,cluster
0,MaSp1a,22.432921,17.814666,23.677962,evm-000033F-processed-gene-27.15-GeneA,000033F,6104352,6121610,"Major, ampullate, spidroin, 1, variant, 1, (Fr...",A0A2S2B4B0,...,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...",LASC004271,"ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_..."
1,MaSp3a,13.046241,14.304353,16.506424,evm-000033F-processed-gene-27.15-GeneD,000033F,6474253,6482733,"Major, ampullate, spidroin, 1, variant, 1, (Fr...",A0A2S2B4B0,...,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...",LASC004275,"ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_..."
2,MaSp1b,11.515409,7.703781,12.856568,evm-000033F-processed-gene-27.15-GeneB,000033F,6189272,6198799,"Major, ampullate, spidroin, 1, variant, 1, (Fr...",A0A2S2B4B0,...,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...",LASC004273,"ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_..."
3,MaSp3b,7.154938,8.087669,8.801191,evm-000033F-processed-gene-27.15-GeneE,000033F,6540284,6549234,"Major, ampullate, spidroin, 1, variant, 1, (Fr...",A0A2S2B4B0,...,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...",LASC004276,"ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_..."
4,MaSp2c,5.650715,5.309158,5.080537,evm-000033F-processed-gene-24.21-GeneA,000033F,5776789,5819345,"MaSp-f(1)#PRD27696.1, protein",PRD27696.1,...,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...",LASC004269,"ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_..."
5,MaSp2f,4.156333,3.800549,4.504349,maker-000175F-exonerate_protein2genome-gene-8....,000175F,1857400,1874685,"Major, ampullate, spidroin, 2, variant, 2, (Fr...",A0A2S2B4G1,...,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...",LASC012922,"ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_..."
6,MaSp1c,3.449324,1.709131,4.009279,evm-000033F-processed-gene-27.15-GeneC,000033F,6258576,6268628,"Major, ampullate, spidroin, 1, variant, 1, (Fr...",A0A2S2B4B0,...,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...",LASC004274,"ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_..."
7,MaSp2b,3.636470,3.133324,3.725936,evm-000033F-processed-gene-24.21-GeneB,000033F,5626769,5627335,"MaSp-f(1)#PRD27696.1, protein",PRD27696.1,...,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...",LASC004267,"ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_..."
8,MaSp2e,2.859585,1.910398,3.031373,maker-000175F-exonerate_protein2genome-gene-8....,000175F,1701320,1716556,"Major, ampullate, spidroin, 2, variant, 2, (Fr...",A0A2S2B4G1,...,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...",LASC012920,"ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_..."
9,MaSp4,0.895281,1.332501,1.085557,evm-000033F-processed-gene-22.11,000033F,5379690,5392488,"MaSp-f(1)#PRD27696.1, protein",PRD27696.1,...,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...",LASC004263,"ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_..."


In [ ]:
prot_gland[prot_gland['geneName'].isin(["gld_15", "pxd_2"])]

,geneName,percent,gene_id,geneNameProtein,silkGroup


In [ ]:
# merged_df.to_csv("C:/Users/46705/Documents/SpiderSilk/data/proteins_18.csv", index = False)


In [ ]:
prot18_["Ortogroup"].unique()

array(['OG0000050', 'OG0009245', 'OG0013923', 'OG0000171', 'OG0002778',
       'OG0007640', 'OG0000494', 'OG0000264', 'OG0006927'], dtype=object)

In [ ]:
prot18_

,lascID,Ortogroup,geneNameProtein
0,LASC004271,OG0000050,MaSp1a
1,LASC004275,OG0000050,MaSp3a
2,LASC004273,OG0000050,MaSp1b
3,LASC004269,OG0000050,MaSp2c
4,LASC004274,OG0000050,MaSp1c
5,LASC004276,OG0000050,MaSp3b
6,LASC004267,OG0000050,MaSp2b
7,LASC012922,OG0000050,MaSp2f
8,LASC012920,OG0000050,MaSp2e
9,LASC013229,OG0009245,SpiCE-LMa1


In [ ]:
silk_top_20 = []
for i, df in enumerate(df_list): 
    df_20 = df.head(20)
    merged_df = pd.merge(df_20, prot18_, how='inner', left_on='OG_id', right_on='Ortogroup')
    silk_top_20.append(merged_df)

df_list = [ara_mag_sum, mag_ara_bool_1, mag_ara_bool_2, ara_mag_GC]
silk_top_20[3]

,OG_id,PC1,PC2,PC3,PC4,PC5,Group,GeneName,Product,Lascid,cluster,lascID,Ortogroup,geneNameProtein
0,OG0000050,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...","LASC004263,LASC004265,LASC004267,LASC004269,LA...","ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_...",LASC004271,OG0000050,MaSp1a
1,OG0000050,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...","LASC004263,LASC004265,LASC004267,LASC004269,LA...","ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_...",LASC004275,OG0000050,MaSp3a
2,OG0000050,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...","LASC004263,LASC004265,LASC004267,LASC004269,LA...","ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_...",LASC004273,OG0000050,MaSp1b
3,OG0000050,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...","LASC004263,LASC004265,LASC004267,LASC004269,LA...","ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_...",LASC004269,OG0000050,MaSp2c
4,OG0000050,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...","LASC004263,LASC004265,LASC004267,LASC004269,LA...","ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_...",LASC004274,OG0000050,MaSp1c
5,OG0000050,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...","LASC004263,LASC004265,LASC004267,LASC004269,LA...","ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_...",LASC004276,OG0000050,MaSp3b
6,OG0000050,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...","LASC004263,LASC004265,LASC004267,LASC004269,LA...","ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_...",LASC004267,OG0000050,MaSp2b
7,OG0000050,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...","LASC004263,LASC004265,LASC004267,LASC004269,LA...","ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_...",LASC012922,OG0000050,MaSp2f
8,OG0000050,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...","LASC004263,LASC004265,LASC004267,LASC004269,LA...","ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_...",LASC012920,OG0000050,MaSp2e
9,OG0000050,-0.031896,0.120426,0.195810,-0.075735,0.154411,Spidroin,"MaSp3b, MaSp2f, MaSp2d, MaSp3a, MaSp2c, MiSpb,...","MaSp-f(1)#PRD27696.1, protein,Major, ampullate...","LASC004263,LASC004265,LASC004267,LASC004269,LA...","ZoneB_MaSp3, nan, ZoneABC, ZoneA_MaSp1, ZoneA_...",LASC004263,OG0000050,MaSp4


In [ ]:
# import pandas as pd
# from scipy.stats import f_oneway

# # Example DataFrames

# # Calculate fractions
# fraction_df = ara_mag_sum_exploded[ara_mag_sum_exploded['Lascid'].isin(prot_silk['lascID'])].shape[0] / ara_mag_sum_exploded.shape[0]
# fraction_df_part = ara_mag_sum_20_exploded[ara_mag_sum_20_exploded['Lascid'].isin(prot_silk['lascID'])].shape[0] / ara_mag_sum_20_exploded.shape[0]

# # Perform ANOVA test
# f_value, p_value = f_oneway([fraction_df], [fraction_df_part])
# print(p_value)

# # Interpret the result
# alpha = 0.05
# if p_value < alpha:
#     print("There is a significant difference in the fraction of values between the DataFrames.")
# else:
#     print("There is no significant difference in the fraction of values between the DataFrames.")


In [ ]:
mag

,Orthogroup,ID,idv_id,DRR,species_x,Genus,Family,OG0000013,OG0000022,OG0000031,...,five_percent_weightloss,ten_percent_weightloss,crystallinity,birefringence,birefringence_sd,diameter,diameter_sd,water_content,supercontraction,supercontraction_sd
0,Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,geneName,NaN,NaN,NaN,NaN,NaN,NaN,"cyp4c3_3,cyp4v2_cyp4c3_5,cyp4v2_23,cyp4v2_3,cy...","LASC002598,LASC010177,LASC010178,LASC010181,LA...","jhamt_16,LASC005293,jhamt_8,jhamt_11,jhamt_5,j...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,product,NaN,NaN,NaN,NaN,NaN,NaN,"Cytochrome P450 4c3,Cytochrome P450 4V2,Cytoch...",hypothetical protein,"Juvenile hormone acid O-methyltransferase,hypo...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,lascID,NaN,NaN,NaN,NaN,NaN,NaN,"LASC003001,LASC003994,LASC003995,LASC005732,LA...","LASC002598,LASC010177,LASC010178,LASC010181,LA...","LASC005289,LASC005293,LASC007879,LASC007882,LA...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,101-W7_S13,101.0,DRR296821,pilipes,Nephila,Araneidae,8.071637352461016,8.646158335980173,9.98759718640447,...,NaN,NaN,26.0,39.6,10.4,2.75,0.09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,282,7416-W1_S13,7416.0,DRR168635,-,Cyrtophora,Araneidae,6.685198909399794,5.3871184863428105,8.271033619828959,...,293.0,317.0,28.0,54.3,2.7,2.49,0.06,3.6,NaN,NaN
135,283,7421-W1_S15,7421.0,DRR168623,extrusa,Caerostris,Araneidae,7.256716041040335,7.211023294093093,9.12808312070015,...,NaN,NaN,23.0,45.3,2.1,1.45,0.04,NaN,35.2,11.1
136,284,7426-W1_S6,7426.0,DRR168628,-,Acrosomoides,Araneidae,4.737890606089994,3.446384610800192,7.49681745895526,...,NaN,NaN,25.0,33.6,2.0,2.23,0.04,NaN,39.3,5.3
137,285,7439-W1_S16,7439.0,DRR168624,darwini,Caerostris,Araneidae,8.635579390054453,7.468948571055397,9.36551228527911,...,268.0,290.0,31.0,42.5,1.2,3.44,0.03,7.1,43.9,4.4
